In [3]:
import pandas as pd
import numpy as np
from sklearn_pandas import DataFrameMapper, gen_features
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

## Read data
We are not reading the original data from Kaggel.  
The data we are reading is based on the data from Kaggle with small changes:
* Random replacement of values with NaNs
* Transformations of categorical (int) features to text categories

In [4]:
!ls data/raw/heart.csv

data/raw/heart.csv


In [10]:
np.random.seed(seed=42)
df_data = pd.read_csv("data/raw/heart.csv")#, sep=';', index_col="id")
# display(df_data.describe())#include="all"))
# display(df_data.head())

## Initializations 

Declerations of all column types and target column

In [11]:
category_features = [["cp"], ['restecg'], ['exang'], ['thal'], ['ca'], ['slope']]
binary_features = [["sex"], ["fbs"]]
numeric_features = [["age"], ["trestbps"], ["chol"], ["thalach"], ["oldpeak"]]
target = "target"

categorical_suffix = "_#CAT#"

Split the data into features and labels

In [12]:
X = df_data.copy()
y = X.pop(target)

## Split data to train test datasets

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Custom Transformers

We create custom transformers for feature engineering

### Blood Pressure Transformer

Custom transformer responsible for the creation of a new blood pressure categorical feature based on systolic (the number at the top) and diastolic (the number at the bottom) values.  
The transformer will create a new categorical feature with values according to the American Heart Association ranges of blood pressure:
* normal  
* elevated  
* high_pressure_stage_1  
* high_pressure_stage_2  
* hypertensive_crisis  
  
  
![title](https://www.inquirer.com/resizer/jACzP-uAGFwY4JBAfo7tY6qUj-k=/1400x932/smart/arc-anglerfish-arc2-prod-pmn.s3.amazonaws.com/public/PHXNJPTEGRB4RCDYCFJZ6IBOZY.jpg)  
Photo from [American Heart Association](https://www.heart.org/-/media/data-import/downloadables/pe-abh-what-is-high-blood-pressure-ucm_300310.pdf?la=en&hash=CAC0F1D377BDB7BC3870993918226869524AAC3D)

In [ ]:
class BloodPressureTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        # Systolic and diastolic blood pressure ranges based on the American Heart Association
        self.systolic_ranges = [-np.inf, 119, 129, 139, 180, np.inf]
        self.diastolic_ranges = [-np.inf, 79, 89, 120, np.inf]
        
        # Blood pressure categories
        self.blood_pressure_category = ["normal", "elevated", "high_pressure_stage_1", "high_pressure_stage_2", "hypertensive_crisis"]
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Copy the data so we will not change the original instance
        df_blood_pressure = X.copy()
        
        # Break down ranges of systolic values to categories
        df_blood_pressure["systolic"] = pd.cut(df_blood_pressure["ap_hi"], self.systolic_ranges, labels=["<120", "120-129", "130-139", "140-180", ">180"])
        
        # Break down ranges of diastolic values to categories
        df_blood_pressure["diastolic"] = pd.cut(df_blood_pressure["ap_lo"], self.diastolic_ranges, labels=["<79", "80-89", "90-120", ">120"])
        
        # Combine ranges from systolic and diastolic features to determine the category of the blood pressure feature
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == "<120") &
                              (df_blood_pressure["diastolic"] == "<79"), "blood_pressure"] = self.blood_pressure_category[0]
        
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == "120-129") &
                              (df_blood_pressure["diastolic"] == "<79"), "blood_pressure"] = self.blood_pressure_category[1]
        
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == "130-139") |
                              (df_blood_pressure["diastolic"] == "80-89"), "blood_pressure"] = self.blood_pressure_category[2]
        
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == "140-180") |
                              (df_blood_pressure["diastolic"] == "90-120"), "blood_pressure"] = self.blood_pressure_category[3]
        
        df_blood_pressure.loc[(df_blood_pressure["systolic"] == ">180") |
                              (df_blood_pressure["diastolic"] == ">120"), "blood_pressure"] = self.blood_pressure_category[4]
        
        # Return blood pressure feature as a dataframe with one column
        return df_blood_pressure[["blood_pressure"]]

### Unhealty Lifestyle Transformer

Custom transformer responsible for the creation of a new "unhealty lifestyle" feature.  
This is a boolean feature representing the use of cigarettes, alcohol, and physical inactivity. 

In [ ]:
class UnhealtyLifestyleTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Copy the data so we will not change the original instance
        df_unhealty_lifestyle = X.copy()
        
        # If you smoke or use alcohol or don't do physical activity, you maintain an unhealty lifestyle!
        df_unhealty_lifestyle["unhealty_lifestyle"] = df_unhealty_lifestyle["smoke"] | df_unhealty_lifestyle["alco"] | (1 - df_unhealty_lifestyle["active"])
        
        # Return unhealty lifestyle feature as a dataframe with one column
        return df_unhealty_lifestyle[["unhealty_lifestyle"]]

## Definition of DataFrameMapper transformers

Now we will define the pipeline of transformations and the raw features we need to complete the creation and processing of the new features and the original features.  
We will pass this to the DataFrameMapper class of the sklearn-pandas package.

In [ ]:
# Input features "ap_hi", "ap_lo".
# Steps:
#    BloodPressureTransformer - create blood pressure feature based on "ap_hi", "ap_lo".
#    SimpleImputer - fill nans with the most frequent value.
#    OneHotEncoder - encode categorical values as a one-hot numeric array.
gen_blood_pressure = (
    ["ap_hi", "ap_lo"],
    [
        BloodPressureTransformer(),
        SimpleImputer(strategy="most_frequent")
    ],
    {"alias": f"blood_pressure{categorical_suffix}"}
)

gen_blood_pressure

(['ap_hi', 'ap_lo'],
 [BloodPressureTransformer(),
  SimpleImputer(copy=True, fill_value=None, missing_values=nan,
         strategy='most_frequent', verbose=0)],
 {'alias': 'blood_pressure_#CAT#'})

In [ ]:
# Input features ["smoke", "alco", "active"].
# Steps:
#    UnhealtyLifestyleTransformer - create unhealty lifestyle feature based on "smoke", "alco", "active".
#    SimpleImputer - fill nans with the most frequent value.
gen_unhealty_lifestyle = (
    ["smoke", "alco", "active"],
    [
        UnhealtyLifestyleTransformer(),
        SimpleImputer(strategy="most_frequent")
    ],
    {"alias": f"unhealty_lifestyle{categorical_suffix}"}
)

gen_unhealty_lifestyle

(['smoke', 'alco', 'active'],
 [UnhealtyLifestyleTransformer(),
  SimpleImputer(copy=True, fill_value=None, missing_values=nan,
         strategy='most_frequent', verbose=0)],
 {'alias': 'unhealty_lifestyle_#CAT#'})

### Apply the same transformers for multiple columns with gen_features

In [ ]:
# Input features [["cholesterol"], ["gluc"]] (The columns are now list of lists because we want to send 2-dimentional DataFrame to each of the transformers).
# Steps:
#    SimpleImputer - fill nans with the most frequent value.
#    OneHotEncoder - encode categorical values as a one-hot numeric array.
gen_category = gen_features(
    columns=category_features,
    classes=[
        {
            "class": SimpleImputer,
            "strategy": "most_frequent"
        }
    ]
)

gen_category = [(col_name, transformer, {"alias": col_name[0] + categorical_suffix}) for col_name, transformer in gen_category]
gen_category

[(['cholesterol'],
  [SimpleImputer(copy=True, fill_value=None, missing_values=nan,
          strategy='most_frequent', verbose=0)],
  {'alias': 'cholesterol_#CAT#'}),
 (['gluc'], [SimpleImputer(copy=True, fill_value=None, missing_values=nan,
          strategy='most_frequent', verbose=0)], {'alias': 'gluc_#CAT#'})]

In [ ]:
# Input features [["gender"], ["smoke"], ["alco"], ["active"]] (The columns are now list of lists because we want to send 2-dimentional DataFrame to each of the transformers).
# Steps:
#    SimpleImputer - fill nans with the most frequent value.
gen_binary = gen_features(
    columns=binary_features,
    classes=[
        {
            "class": SimpleImputer,
            "strategy": "most_frequent"
        }
    ]
)

gen_binary = [(col_name, transformer, {"alias": col_name[0] + categorical_suffix}) for col_name, transformer in gen_binary]
gen_binary

[(['gender'], [SimpleImputer(copy=True, fill_value=None, missing_values=nan,
          strategy='most_frequent', verbose=0)], {'alias': 'gender_#CAT#'}),
 (['smoke'], [SimpleImputer(copy=True, fill_value=None, missing_values=nan,
          strategy='most_frequent', verbose=0)], {'alias': 'smoke_#CAT#'}),
 (['alco'], [SimpleImputer(copy=True, fill_value=None, missing_values=nan,
          strategy='most_frequent', verbose=0)], {'alias': 'alco_#CAT#'}),
 (['active'], [SimpleImputer(copy=True, fill_value=None, missing_values=nan,
          strategy='most_frequent', verbose=0)], {'alias': 'active_#CAT#'})]

In [14]:
import numpy as np
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit([[7, 2, 3], [4, np.nan, 6], [10, 5, 9]])

X = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
print(imp_mean.transform(X))

[[ 7.   2.   3. ]
 [ 4.   3.5  6. ]
 [10.   3.5  9. ]]


In [ ]:
# Input features [["age"], ["height"], ["weight"], ["ap_hi"], ["ap_lo"]] (The columns are now list of lists because we want to send 2-dimentional DataFrame to each of the transformers).
# Steps:
#    SimpleImputer - fill nans with the mean value.
#    StandardScaler - standardize features by removing the mean and scaling to unit variance.
gen_numeric = gen_features(
    columns=numeric_features,
    classes=[
        {
            "class": SimpleImputer,
            "strategy": "mean"
        },
        {
            "class": StandardScaler
        }
    ]
)

gen_numeric

[(['age'],
  [SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
          verbose=0),
   StandardScaler(copy=True, with_mean=True, with_std=True)]),
 (['height'],
  [SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
          verbose=0),
   StandardScaler(copy=True, with_mean=True, with_std=True)]),
 (['weight'],
  [SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
          verbose=0),
   StandardScaler(copy=True, with_mean=True, with_std=True)]),
 (['ap_hi'],
  [SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
          verbose=0),
   StandardScaler(copy=True, with_mean=True, with_std=True)]),
 (['ap_lo'],
  [SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
          verbose=0),
   StandardScaler(copy=True, with_mean=True, with_std=True)])]

### DataFrameMapper Construction

Now we will define the course of action of the DataFrameMapper and indicate that the input and output will be Pandas Dataframe.

In [ ]:
preprocess_mapper = DataFrameMapper(
    [
        gen_blood_pressure,
        gen_unhealty_lifestyle,
        *gen_category,
        *gen_binary,
        *gen_numeric,
    ],
    input_df=True,
    df_out=True
)

In [ ]:
preprocess_mapper.fit_transform(X_train, y_train).head()

,blood_pressure_#CAT#,unhealty_lifestyle_#CAT#,cholesterol_#CAT#,gluc_#CAT#,gender_#CAT#,smoke_#CAT#,alco_#CAT#,active_#CAT#,age,height,weight,ap_hi,ap_lo
id,,,,,,,,,,,,,
98125,high_pressure_stage_2,0,well_above_normal,normal,women,0,0,1,0.388175,-0.549199,1.024035e-15,-0.056261,-0.035475
28510,high_pressure_stage_1,1,well_above_normal,well_above_normal,men,0,0,0,1.308468,0.333478,-6.621198e-01,-0.056261,-0.085423
15795,high_pressure_stage_2,0,normal,normal,women,0,0,1,1.346527,-0.549199,-5.900598e-01,-0.056261,-0.035475
39560,high_pressure_stage_2,0,well_above_normal,normal,women,0,0,1,1.291463,-0.170909,-1.382720e+00,-0.024632,-0.035475
32677,high_pressure_stage_2,0,well_above_normal,normal,women,0,0,1,0.912495,-0.801392,7.790809e-01,0.133513,-0.085423


### Custom CatBoost Classifier

We need to implement our own catboost classifier so we can track our categorical features at runtime

In [ ]:
class CustomCatBoostClassifier(CatBoostClassifier):

    def fit(self, X, y=None, **fit_params):
        print(X.filter(regex=f"{categorical_suffix}$").columns.to_list())

        return super().fit(
            X,
            y=y,
            cat_features=X.filter(regex=f"{categorical_suffix}$").columns,
            **fit_params
        )

### Feature Selection

In the feature selection step, we create out own custom feature selection so the input dataframe will stay a dataframe and not numpy array

In [ ]:
class CustomFeatureSelection(SelectFromModel):

    def transform(self, X):
        
        # Get indices of important features
        important_features_indices = list(self.get_support(indices=True))

        # Select important features
        _X = X.iloc[:, important_features_indices].copy()

        return _X

### Sklearn Pipeline

Now we piece together all prevoius definitions to define the full pipeline:
* preprocessing
* feature selection
* estimator

In [ ]:
pipeline = Pipeline(steps=[
    ("preprocess", preprocess_mapper),
    ("feature_selection", CustomFeatureSelection(CustomCatBoostClassifier(logging_level="Silent"))),
    ("estimator", CustomCatBoostClassifier(logging_level="Silent"))
])

In [ ]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

['blood_pressure_#CAT#', 'unhealty_lifestyle_#CAT#', 'cholesterol_#CAT#', 'gluc_#CAT#', 'gender_#CAT#', 'smoke_#CAT#', 'alco_#CAT#', 'active_#CAT#']


## Grid Search

Next I created a grid search object which includes the original pipeline.  
When I then call fit, the transformations are applied to the data, before a cross-validated grid-search is performed over the parameter grid.

In [ ]:
pipeline = Pipeline(steps=[
    ("preprocess", preprocess_mapper),
    ("feature_selection", CustomFeatureSelection(CustomCatBoostClassifier(logging_level="Silent"))),
    ("estimator", CustomCatBoostClassifier(logging_level="Silent"))
])

In [ ]:
param_grid = {
    "estimator__depth": [4, 5, 6],
    "estimator__iterations": [500, 1000],
    "estimator__learning_rate": [0.001, 0.01, 0.1], 
    "estimator__l2_leaf_reg": [3, 5, 100],
    "estimator__border_count": [10, 50, 200],
    "estimator__ctr_border_count": [10, 50, 200],
    "estimator__thread_count": [4]
}

In [ ]:
gscv_estimator = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
gscv_estimator.fit(X_train, y_train)

In [ ]:
display(gscv_estimator.best_params_)

In [ ]:
preds = gscv_estimator.predict(X_test)